In [1]:
from youtube_transcript_api import YouTubeTranscriptApi

transcript_as_list = YouTubeTranscriptApi.get_transcript("HYVeP4F0GNU")

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import json

In [4]:
import os
os.environ["OCTOAI_TOKEN"] = os.getenv("OCTOAI_API_TOKEN")

In [5]:
from octoai.client import Client

client = Client()

In [6]:
transcript_string = ""
for entry in transcript_as_list:
    transcript_string += entry['text']+ " "

In [7]:
len(transcript_string)

154763

In [8]:
len(transcript_as_list)

4095

In [9]:
transcripts_chunked = []
current_chunk = " "
for entry in transcript_as_list:
    entry_text = entry["text"] + " "
    if len(current_chunk) + len(entry_text) <= 8192:
        current_chunk += entry_text
    else:
        transcripts_chunked.append(current_chunk[:-1])
        current_chunk = entry_text
transcripts_chunked.append(current_chunk)

In [10]:
len(transcripts_chunked)

19

In [11]:
def get_summary(snippet: str) -> dict:

    completion = client.chat.completions.create(
    messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant. Summarize the user input. Keep your responses limited to one short paragraph if possible."
            },
            {
                "role": "user",
                "content": f"{snippet}"
            }
        ],
        model="nous-hermes-2-mixtral-8x7b-dpo",
        max_tokens=7500,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
    )
    return completion.dict()

In [12]:
summaries = []
for chunk in transcripts_chunked:
    octo_output = get_summary(chunk)
    summaries.append(octo_output["choices"][0]["message"]["content"])

In [13]:
summaries

['In this podcast, Dr. Andrew Huberman interviews Dr. Mark Desposito, a neurologist and professor of neuroscience and psychology at the University of California, Berkeley. They discuss executive function, memory, and the neural circuits controlling them. They also explore the role of dopamine in executive function and working memory, as well as ways to optimize cognitive function in both healthy and diseased or injured brains. The podcast is sponsored by Maui Nui Venison, JOOVV Red Light Therapy, and 8 Sleep Smart Mattress Covers.',
 'The user discussed the prefrontal cortex, also known as the frontal lobe, which is responsible for higher-level cognitive abilities and executive functions such as planning, organizing, and cognitive control. They mentioned that the frontal lobe is the last part of the brain to fully develop, around the early 20s, and its slow development allows for exploration and flexibility. The frontal lobe is divided into subregions, with the lateral portion involved

In [15]:
full_len = sum(len(summaries[i]) for i in range(len(summaries)))

In [16]:
full_len

13166

In [19]:
# no idea how long of a token string this is
full_snippet = " ".join(summaries)
summarized_summary = get_summary(full_snippet)["choices"][0]["message"]["content"]

In [20]:
summarized_summary

"In this podcast, Dr. Andrew Huberman interviews Dr. Mark Desposito on executive function, memory, and neural circuits. They discuss the role of dopamine in executive function and working memory, as well as ways to optimize cognitive function in healthy and diseased or injured brains. The conversation covers topics such as the prefrontal cortex, cognitive therapy, dopamine's role in memory and movement, measuring dopamine levels, dopamine agonists, concussions, brain health, Alzheimer's and Parkinson's diseases, sex differences in dopamine levels, mindfulness meditation, and psychedelics. They also explore the importance of understanding brain network activity and action states, and the potential use of psychedelics like Sil cybin for depression treatment."

In [20]:
# import json
# print(json.dumps(completion.dict(), indent=2))

{
  "id": "cmpl-78b97af3d0c84442a1c5b4b5452b86c6",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The Huberman Lab podcast discusses science and science-based tools for everyday life, with a focus on executive function and memory. The guest, Dr. Mark Desposito, is a neurologist and professor of neuroscience and psychology at the University of California, Berkeley. He explains the neural circuits controlling executive function and memory, the role of dopamine in executive function and working memory, and ways to optimize cognitive function. The discussion also covers restoring cognitive function in disease or injury conditions such as traumatic brain injury, concussion, Alzheimer's, Parkinson's, and attention deficit disorders. The podcast is sponsored by Maui Nei venison and JuJu medical-grade red light therapy devices.",
        "function_call": null
      },
      "delta": null,
      "finish_reason": "stop"
    }
  ],
  "cr